In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from source.read_extrasensory import readAcc

In [2]:
X, y, dimensions = readAcc()
X = X.transpose([0, 2, 1])

In [13]:
X.shape

(70000, 100, 3)

In [3]:
def filter_dimensions(X, all_dimensions, dimensions):
    idx = [np.where(all_dimensions == dim)[0][0] for dim in dimensions]
    return X[:, :, idx]

def folding_v2(X):
    N, T, D = X.shape
    X_f = np.zeros((N, D * T))
    for n in range(N):
        for t in range(T):
            for d in range(D):
                X_f[n, D * t + d] = X[n, t, d]
    return X_f

# print(X.shape)
# print(X_f.shape)
# features = folding_v2(X_f)
# print(features.shape)
# X_f = X_f.transpose([0, 2, 1])


In [4]:
X.shape

(100000, 100, 3)

In [5]:
N, T, D = X.shape
for n in range(N):
    for d in range(D):
        X[n, :, d] = X[n, :, d] - X[n, :, d].mean()
        
features = folding_v2(X)

In [6]:
features.shape

(100000, 300)

In [7]:
import umap

reducer = umap.UMAP(n_components=2, metric='braycurtis')
# coords = reducer.fit_transform(features)
coords = reducer.fit_transform(features, y=y)

/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [8]:
import sys
sys.path.insert(0, '/home/texs/Documentos/Repositories/mts_viz')

from server.source.storage import MTSStorage

In [9]:
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6, 45, 75, 98, 101, 102], dtype=object)

In [10]:
storage = MTSStorage('extraSensory')
storage.load()

# activities_map = {act: str(act) for act in np.unique(y)}
activities = ['LYING_DOWN', 'SITTING', 'STANDING_IN_PLACE', 'STANDING_AND_MOVING', 'WALKING', 'RUNNING', 'BICYCLING']
activities_map = {i: activities[i] for i in range(len(activities))}
storage.add_mts(
    'acc_bray', X, 
    dimensions, 
    coords={'umap': coords}, 
    labels={'activities': y}, 
    labelsNames={'activities': activities_map},
    sampling = True,
    n_samples = 1000
)
storage.save()

In [10]:
X.shape

(70000, 3, 100)